In [1]:
import netket as nk
import numpy as np
import matplotlib.pyplot as plt
import json

from netket import experimental as nkx
import netket.nn as nknn
import netket.optimizer as nkopt
import jax
import jax.numpy as jnp

from sympy import Matrix

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [2]:
# create graph

x = 2  # take a 2x4 lattice
y = 4

# create the graph our fermions can hop on
g = nk.graph.Grid(extent=[x, y], pbc=False)
n_sites = g.n_nodes

g

Lattice(
    n_nodes=8,
    extent=[2 4],
    basis_vectors=
        [[1. 0.]
         [0. 1.]],
    site_offsets=
        [[0. 0.]],
)

In [3]:
hi = nkx.hilbert.SpinOrbitalFermions(n_sites, n_fermions=2)
hi

SpinOrbitalFermions(n_orbitals=8, n_fermions=2)

In [5]:
# create an operator representing fermi hubbard interactions
# -t (i^ j + h.c.) + U (i^ i j^ j)
# we will create a helper function to abbreviate the creation, destruction and number operators
# each operator has a site and spin projection (sz) in order to find the right position in the hilbert space samples
def c(site):
    return nkx.operator.fermion.destroy(hi, site)
    
def cdag(site):
    return nkx.operator.fermion.create(hi, site)

def nc(site):
    return nkx.operator.fermion.number(hi, site)

In [97]:
t = 100  # tunneling/hopping
U = 0.01  # coulomb

hamkin = 0.0
hampot = 0.0
for u, v in g.edges():
    print(u, v)
    hamkin += -t * (cdag(u) * c(v)) + cdag(v) * c(u))

    hampot += U * nc(u) * nc(v)
ham = hamkin + hampot

0 1
1 2
0 4
1 5
3 7
2 3
6 7
4 5
2 6
5 6


In [98]:
print("Hamiltonian =", ham.operator_string())

Hamiltonian = -100.0 [1^ 0] +
-100.0 [2^ 1] +
-100.0 [4^ 0] +
-100.0 [5^ 1] +
-100.0 [7^ 3] +
-100.0 [3^ 2] +
-100.0 [7^ 6] +
-100.0 [5^ 4] +
-100.0 [6^ 2] +
-100.0 [6^ 5] +
0.01 [0^ 0 1^ 1] +
0.01 [1^ 1 2^ 2] +
0.01 [0^ 0 4^ 4] +
0.01 [1^ 1 5^ 5] +
0.01 [3^ 3 7^ 7] +
0.01 [2^ 2 3^ 3] +
0.01 [6^ 6 7^ 7] +
0.01 [4^ 4 5^ 5] +
0.01 [2^ 2 6^ 6] +
0.01 [5^ 5 6^ 6]


In [99]:
Matrix(ham.to_dense())

Matrix([
[0.01, -100.0,      0,      0,      0,      0,    0,  100.0,      0,      0, -100.0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0],
[   0,      0, -100.0, -100.0,      0,      0,    0,      0,  100.0,      0,      0,      0,      0,      0,      0, -100.0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0],
[   0,      0,   0.01,      0, -100.0,      0,    0,      0,      0,      0,      0,      0,  100.0,      0,      0,      0, -100.0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0],
[   0,      0,      0,      0, -100.0,      0,    0,      0,      0,  100.0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0,      0, -100.0,      0,      0,      0,      0,      0,      0],
[   0,      0,      0,      0,      0, -100.0,    0,      0,      0,      0,      0,      0,      0,  1

In [100]:
occupation = np.zeros(n_sites)
occupation[0] = occupation[1] = 1
res = ham.get_conn(occupation)
res

(array([[1., 1., 0., 0., 0., 0., 0., 0.]]), array([0.01]))

In [105]:
Hdense = ham.to_dense()
Hdense_kin = hamkin.to_dense()
Hdense_pot = hampot.to_dense()

occupation = np.zeros(n_sites)
occupation[0] = 1
occupation[2] = 1
#occupation[2] = occupation[3] = 1
#occupation[4] = occupation[5] = 0.25
#occupation[6] = occupation[7] = 0.25

i = hi.states_to_numbers(occupation)
v = np.zeros(hi.n_states)
v[i] = 1

v /= np.linalg.norm(v)

In [106]:
occupation = np.zeros(n_sites)
occupation[0] = occupation[2] = 1
i = hi.states_to_numbers(occupation)
w = np.zeros(hi.n_states)
w[i] = 1

w /= np.linalg.norm(w)

E = (w.conj().T).dot(Hdense.dot(v))
E

0.0

In [107]:
E = (v.conj().T).dot(Hdense_kin.dot(v))
E

0.0

In [108]:
E = (v.conj().T).dot(Hdense_pot.dot(v))
E

0.0

## Evolution

In [23]:
from netket.experimental import TDVP
from netket.experimental.dynamics import RK23
from netket.vqs import VariationalState

In [36]:
integrator = RK23(dt=0.01, adaptive=True, rtol=1e-3, atol=1e-3)

i = hi.states_to_numbers(occupation)
occupation = np.zeros(n_sites)
occupation[0] = occupation[1] = 1
state = VariationalState(i)

driver = TDVP(
    operator=ham,
    variational_state=state,
    integrator=integrator
)

In [37]:
driver.run(
    T=0.3,
    out="example_dynamics",
    tstops=np.linspace(0.0, 0.3, 3, endpoint=True),
)

  0%|                                                                                                         …

NotImplementedError: odefun not implemented for <class 'netket.vqs.base.VariationalState'>